In [1]:
import googlemaps
import pandas as pd
from datetime import datetime

gmaps = googlemaps.Client(key='YourKey')

from googlemaps import convert
from googlemaps.convert import as_list

def distance_matrix(client, origins, destinations,
                    mode=None, language=None, avoid=None, units=None,
                    departure_time=None, arrival_time=None, transit_mode=None,
                    transit_routing_preference=None, traffic_model=None, region=None):
    """ Gets travel distance and time for a matrix of origins and destinations.
    :param origins: One or more locations and/or latitude/longitude values,
        from which to calculate distance and time. If you pass an address as
        a string, the service will geocode the string and convert it to a
        latitude/longitude coordinate to calculate directions.
    :type origins: a single location, or a list of locations, where a
        location is a string, dict, list, or tuple
    :param destinations: One or more addresses and/or lat/lng values, to
        which to calculate distance and time. If you pass an address as a
        string, the service will geocode the string and convert it to a
        latitude/longitude coordinate to calculate directions.
    :type destinations: a single location, or a list of locations, where a
        location is a string, dict, list, or tuple
    :param mode: Specifies the mode of transport to use when calculating
        directions. Valid values are "driving", "walking", "transit" or
        "bicycling".
    :type mode: string
    :param language: The language in which to return results.
    :type language: string
    :param avoid: Indicates that the calculated route(s) should avoid the
        indicated features. Valid values are "tolls", "highways" or "ferries".
    :type avoid: string
    :param units: Specifies the unit system to use when displaying results.
        Valid values are "metric" or "imperial".
    :type units: string
    :param departure_time: Specifies the desired time of departure.
    :type departure_time: int or datetime.datetime
    :param arrival_time: Specifies the desired time of arrival for transit
        directions. Note: you can't specify both departure_time and
        arrival_time.
    :type arrival_time: int or datetime.datetime
    :param transit_mode: Specifies one or more preferred modes of transit.
        This parameter may only be specified for requests where the mode is
        transit. Valid values are "bus", "subway", "train", "tram", "rail".
        "rail" is equivalent to ["train", "tram", "subway"].
    :type transit_mode: string or list of strings
    :param transit_routing_preference: Specifies preferences for transit
        requests. Valid values are "less_walking" or "fewer_transfers".
    :type transit_routing_preference: string
    :param traffic_model: Specifies the predictive travel time model to use.
        Valid values are "best_guess" or "optimistic" or "pessimistic".
        The traffic_model parameter may only be specified for requests where
        the travel mode is driving, and where the request includes a
        departure_time.
    :param region: Specifies the prefered region the geocoder should search
        first, but it will not restrict the results to only this region. Valid
        values are a ccTLD code.
    :type region: string
    :rtype: matrix of distances. Results are returned in rows, each row
        containing one origin paired with each destination.
    """

    params = {
        "origins": convert.location_list(origins),
        "destinations": convert.location_list(destinations)
    }

    if mode:
        # NOTE(broady): the mode parameter is not validated by the Maps API
        # server. Check here to prevent silent failures.
        if mode not in ["driving", "walking", "bicycling", "transit"]:
            raise ValueError("Invalid travel mode.")
        params["mode"] = mode

    if language:
        params["language"] = language

    if avoid:
        if avoid not in ["tolls", "highways", "ferries"]:
            raise ValueError("Invalid route restriction.")
        params["avoid"] = avoid

    if units:
        params["units"] = units

    if departure_time:
        params["departure_time"] = convert.time(departure_time)

    if arrival_time:
        params["arrival_time"] = convert.time(arrival_time)

    if departure_time and arrival_time:
        raise ValueError("Should not specify both departure_time and"
                         "arrival_time.")

    if transit_mode:
        params["transit_mode"] = convert.join_list("|", transit_mode)

    if transit_routing_preference:
        params["transit_routing_preference"] = transit_routing_preference

    if traffic_model:
        params["traffic_model"] = traffic_model

    if region:
        params["region"] = region

    return client._request("/maps/api/distancematrix/json", params)

In [72]:
condo=pd.read_excel('Condo_info_Combined - Cleaned.xlsx')
train=pd.read_excel('TrainsLatLong.xlsx')

CondoName,LatLongC=condo['Name'],condo['LatLongC']    
StationName,LatLongT=train['StationName'],train['LatLongT']

print(len(CondoName))
print(len(set(CondoName)))
print(len(StationName))

print(CondoName[0],LatLongC[0])
print(StationName[0],LatLongT[0])


12033
452
92
 Life Sukhumvit 48 13.711499992951,100.594751288400
Mo Chit Bts station 13.8024977,100.5538045


In [74]:
c=0
NearestStation=[]

#loop over all condos
while c <= len(CondoName):
    station=""
    dist=9999999
    disttext=""
    time=""
    
    for i in range(0,len(StationName)): #loop check distance from this condo to each station
        now = datetime.now()
        result = distance_matrix(gmaps,LatLongC[c],LatLongT[i],mode="walking", avoid="ferries")
        if result['rows'][0]['elements'][0]['distance']['value'] < dist:
            dist= result['rows'][0]['elements'][0]['distance']['value']
            station= StationName[i]
            disttext= result['rows'][0]['elements'][0]['distance']['text']
            time= result['rows'][0]['elements'][0]['duration']['text']
            print(dist)
    print(str(c)+" The Nearest Train Station for "+CondoName[c]+ " is "+station,disttext,time+" on foot.")
    NearestStation.append([station,disttext,time])
    c+=1
    
    try:
        while CondoName[c]==CondoName[c-1]: # if the next row is the same condo, copy over information. Exit loop condo name mismatch.
            NearestStation.append([station,disttext,time])
            print(str(c)+" The Nearest Train Station for "+CondoName[c]+ " is "+station,disttext,time+" on foot.")
            c+=1
        print('============= Next Condo') 
    except: break
        
print(NearestStation)
    

14244
13366
11681
10815
9733
9514
8924
7923
6274
5529
3547
2411
1501
687


KeyboardInterrupt: 

In [52]:
output=distance_matrix(gmaps,LatLongC[0] ,LatLongT[0],mode="walking", avoid="ferries")

In [53]:
output


{'destination_addresses': ['39 1, Khwaeng Chatuchak, Khet Chatuchak, Krung Thep Maha Nakhon 10900, Thailand'],
 'origin_addresses': ['Soi Sukhumvit 48, Khwaeng Phra Khanong, Khet Khlong Toei, Krung Thep Maha Nakhon 10110, Thailand'],
 'rows': [{'elements': [{'distance': {'text': '14.2 km', 'value': 14244},
     'duration': {'text': '2 hours 59 mins', 'value': 10711},
     'status': 'OK'}]}],
 'status': 'OK'}

In [54]:
output['rows']

[{'elements': [{'distance': {'text': '14.2 km', 'value': 14244},
    'duration': {'text': '2 hours 59 mins', 'value': 10711},
    'status': 'OK'}]}]

In [55]:
output['rows'][0]['elements'][0]['distance']['text']

'14.2 km'

In [57]:
output['rows'][0]['elements'][0]['duration']['text']

'2 hours 59 mins'

Result: 278.54558935106695
